# Prototype JulES v1

Check out the ReadMe page on Github for a description of JulES. This is a demo of the prototype implementation of JulES in a stepwise manner. Here are some of the most important steps: 
- Load the TuLiPa and JulES code on parallel processors so that we can run scenarios and subsystems in parallel
- Decide on simulation and scenario parameters
- First time step in the simulation and initializing
    - Load datasets for the European power system/market.
    - Initialize and run price prognosis models
    - Do scenario modelling for the stochastic subsystem models
    - Initialize and run the stochastic subsystem models
    - Initialize and run the market clearing problem
    - Collect results
- Simulate the next time steps
    - Run price prognosis models, scenario modelling, stochastic subsystem models, market clearing and collect results for each time step
- Postprocess, store and plot results

### Import packages

In [ ]:
using Pkg;
# Pkg.update("TuLiPa") # uncomment to update TuLiPa to latest version
# Pkg.develop(path=joinpath(dirname(dirname(pwd())),"TuLiPa")); Pkg.status() # go to development version
# Pkg.undo(); Pkg.status() # go back to main package version
# Pkg.add(url="https://github.com/NVE/TuLiPa.git"); Pkg.status() # alternative go back to latest version

In [ ]:
using DataFrames, Statistics, JSON, Distributed, Clustering, YAML, CSV
#plotlyjs(); uncomment for interactive plots

In [ ]:
config = YAML.load_file(joinpath(dirname(dirname(pwd())), "JulESIO", "config_jules_vassdrag.yml"))
scenarioyear = 1981
datayear = 2021

In [ ]:
# config = YAML.load_file(joinpath(dirname(dirname(pwd())), "JulESIO", "config_jules_solbatteri.yml"))
# scenarioyear = 1991
# datayear = 2024

In [ ]:
# config = YAML.load_file(joinpath(dirname(dirname(pwd())), "JulESIO", "config_jules_prognose.yml"))
# scenarioyear = 1991
# datayear = 2024

In [ ]:
# config = YAML.load_file(joinpath(dirname(dirname(pwd())), "JulESIO", "config_jules_la.yml"))
# scenarioyear = 1981
# datayear = 2025

### Prepare parallell processing - import code on all cores
The problem is simulated on 31 2.20 GHz processors running in parallel. TODO: Test on faster processors.

In [ ]:
const numcores = config["main"]["numcores"]

if nprocs() < numcores
    addprocs(numcores - nprocs())
end

@show nprocs();

In [ ]:
@everywhere using TuLiPa, Dates
# @everywhere include(joinpath(dirname(dirname(dirname(pwd()))),"jgrc/TuLiPa/src/TuLiPa.jl"));

In [ ]:
@everywhere include(joinpath(dirname(pwd()),"src/JulES.jl"));

In [ ]:
function getdataset(config, scenarioyear)
    settings = config[config["main"]["settings"]]

    method = config["main"]["function"]
    if method == "nve_prognosis"
        sti_dataset = joinpath(config["main"]["input"], "static_input")
        weekstart = config["main"]["weekstart"]

        sti_dataset1 = joinpath(config["main"]["input"], "Uke_$weekstart", "input")

        exd = JSON.parsefile(joinpath(sti_dataset1, "exogenprices_prognose1.json"))
        exogen = JulES.getelements(exd, sti_dataset1)

        add = JSON.parsefile(joinpath(sti_dataset, "aggdetd2.json"))
        aggdetd = JulES.getelements(add, sti_dataset)

        ipad = JSON.parsefile(joinpath(sti_dataset1, "tilsigsprognoseragg$scenarioyear.json"))
        agginflow = JulES.getelements(ipad, sti_dataset1)

        thd = JSON.parsefile(joinpath(sti_dataset, "termisk1.json"))
        thermal = JulES.getelements(thd, sti_dataset)

        wsd = JSON.parsefile(joinpath(sti_dataset, "vindsol.json"))
        windsol = JulES.getelements(wsd, sti_dataset)

        trd = JSON.parsefile(joinpath(sti_dataset1, "nett.json"))
        transm = JulES.getelements(trd)

        cod = JSON.parsefile(joinpath(sti_dataset, "forbruk5.json"))
        cons = JulES.getelements(cod, sti_dataset)

        fpd = JSON.parsefile(joinpath(sti_dataset1, "brenselspriser.json"))
        fuel = JulES.getelements(fpd, sti_dataset1)

        nud = JSON.parsefile(joinpath(sti_dataset1, "nuclear.json"))
        nuclear = JulES.getelements(nud, sti_dataset1)

        dse = JSON.parsefile(joinpath(sti_dataset, "tidsserier_detd.json"))
        detdseries = JulES.getelements(dse, sti_dataset)

        dda = JSON.parsefile(joinpath(sti_dataset, "dataset_detd.json"))
        detdstructure = JulES.getelements(dda)

        ipd = JSON.parsefile(joinpath(sti_dataset1, "tilsigsprognoser$scenarioyear.json"))
        inflow = JulES.getelements(ipd, sti_dataset1)

        progelements = vcat(exogen, aggdetd, thermal, windsol, transm, cons, agginflow, fuel, nuclear)
        aggstartmagdict = JSON.parsefile(joinpath(sti_dataset1, "aggstartmagdict.json"), dicttype=Dict{String, Float64})

        if settings["problems"]["onlyagghydro"]
            global detailedelements = elements
            global startmagdict = Dict()
            global detailedrescopl = Dict()
            return Dict("elements" => progelements, "startmagdict" => startmagdict, "aggstartmagdict" => aggstartmagdict, "detailedrescopl" => detailedrescopl)
        else
            global elements = vcat(exogen, detdseries, detdstructure, thermal, windsol, transm, cons, inflow, fuel, nuclear)
            global startmagdict = JSON.parsefile(joinpath(sti_dataset1, "startmagdict.json"))
            global detailedrescopl = JSON.parsefile(joinpath(sti_dataset, "magasin_elspot.json")
            return Dict("elements" => elements, "progelements" => progelements, "startmagdict" => startmagdict, "aggstartmagdict" => aggstartmagdict, "detailedrescopl" => detailedrescopl))
        end
    elseif method == "nve_la"
        sti_thema = joinpath(config["main"]["input"], "datasett", "data_fra_thema")
        sti_vannkraft = joinpath(config["main"]["input"], "datasett", "data_fra_dynmodell")

        tsd = JSON.parsefile(joinpath(sti_thema, "dataset_thema.json"))
        themastructure = JulES.getelements(tsd, sti_thema)
        tst = JSON.parsefile(joinpath(sti_thema, "dataset_thema_excl_hydro_nose.json"))
        themastructure_exl = JulES.getelements(tst, sti_thema)
        tse = JSON.parsefile(joinpath(sti_thema, "tidsserier_thema.json"))
        themaseries = JulES.getelements(tse, sti_thema)

        dse = JSON.parsefile(joinpath(sti_vannkraft, "tidsserier_detd.json"))
        detdseries = JulES.getelements(dse)
        dst = JSON.parsefile(joinpath(sti_vannkraft, "dataset_detd.json"))
        detdstructure = JulES.getelements(dst)

        progelements = vcat(themaseries, themastructure)
        
        if settings["problems"]["onlyagghydro"]
            global detailedrescopl = Dict()
            return Dict("elements" => progelements, "detailedrescopl" => detailedrescopl)
        else
            global elements = vcat(themaseries, themastructure_exl, detdseries, detdstructure)
            global detailedrescopl = JSON.parsefile(joinpath(sti_vannkraft, "magasin_elspot.json"))
            return Dict("elements" => elements, "progelements" => progelements, "detailedrescopl" => detailedrescopl)
        end
    elseif method == "nve_solbatteri"
        elements = DataElement[]

        # Solar, battery and transmission parameters
        transmcap = config["data"]["transmcap"] # MW
        transmeff = config["data"]["transmeff"] # Small loss to avoid unnecessary transfers
        storagecap = config["data"]["storagecap"] # GWh
        chargecap = config["data"]["chargecap"]# MW
        lossbattery = config["data"]["lossbattery"] # the whole loss when the battery charges
        solarcap = config["data"]["solarcap"] # MW

        # Power balances for price areas and transmission
        addexogenbalance!(elements, "PowerBalance_ExternalHub", "Power", "AreaPrice")
        price_path = joinpath(config["main"]["input"], config["data"]["price"])
        df = CSV.read(price_path, DataFrame; header=3, decimal=',', types=Float64)
        df[:,"aar"] = cld.(1:first(size(df)), 2912) .+ 1957
        df[:,"tsnitt"] = rem.(0:(first(size(df))-1), 2912) .+ 1
        df.datetime .= getisoyearstart.(Int.(df.aar)) + Hour.((df.tsnitt.-1)*3) # TODO: Include week 53. Now ignored and flat prices.
        push!(elements, DataElement(TIMEINDEX_CONCEPT,"VectorTimeIndex","AreaPriceProfileIndex",
                Dict("Vector" => df.datetime)))
        push!(elements, DataElement(TIMEVALUES_CONCEPT,"VectorTimeValues","AreaPriceProfileValues",
                Dict("Vector" => df[:,"Vestsyd"].*1000))) # *1000 to go from €/MWh to €/GWh
        push!(elements, getelement(TIMEVECTOR_CONCEPT,"RotatingTimeVector","AreaProfile",
                (TIMEINDEX_CONCEPT,"AreaPriceProfileIndex"),(TIMEVALUES_CONCEPT,"AreaPriceProfileValues")))
        addparam!(elements, "MeanSeriesParam", "AreaPrice", 1.0, "AreaProfile")

        addbalance!(elements, "PowerBalance_HomeHub", "Power")

        addpowertrans!(elements, "PowerBalance_ExternalHub", "PowerBalance_HomeHub", transmcap, transmeff)
        addpowertrans!(elements, "PowerBalance_HomeHub", "PowerBalance_ExternalHub", transmcap, transmeff)

        # Add battery
        addbattery!(elements, "Battery", "PowerBalance_HomeHub", storagecap, lossbattery, chargecap)

        # Add solar production as an RHSTerm
        solar_path = joinpath(config["main"]["input"], config["data"]["solar"]) # profiles from https://www.nve.no/energi/analyser-og-statistikk/vaerdatasett-for-kraftsystemmodellene/
        df = CSV.read(solar_path, DataFrame)
        dfmt = DateFormat("yyyy-mm-dd HH:MM:SS")
        df.Timestamp = DateTime.(df.Timestamp, dfmt)
        @assert issorted(df.Timestamp)
        start = first(df.Timestamp)
        numperiods = length(df.Timestamp)
        push!(elements, DataElement(TIMEINDEX_CONCEPT, "RangeTimeIndex", "SolProfileTimeIndex", 
                Dict("Start" => start, "Delta" => Hour(1), "Steps" => numperiods)))
        push!(elements, DataElement(TIMEVALUES_CONCEPT, "VectorTimeValues", "SolProfilValues",
                Dict("Vector" => df.SolarGER)))
        push!(elements, DataElement(TIMEVECTOR_CONCEPT, "RotatingTimeVector", "SolProfil",
                Dict(TIMEVALUES_CONCEPT => "SolProfilValues", TIMEINDEX_CONCEPT => "SolProfileTimeIndex")))
        push!(elements, DataElement(PARAM_CONCEPT, "MWToGWhSeriesParam", "SolParam", Dict("Level" => solarcap, "Profile" => "SolProfil")))
        addrhsterm!(elements, "SolParam", "PowerBalance_HomeHub", DIRECTIONIN)

        return Dict("elements" => elements, "detailedrescopl" => Dict())
    elseif method == "nve_vassdrag"
        # Read watercourse, elspot area and price series
        watercourse = config["data"]["watercourse"]
        elspotnames = config["data"]["elspotnames"] # some watercourses are in several elspot areas
        priceseriesname = config["data"]["priceseriesname"] 

        # Read hydropower dataelements from json-files
        sti_dynmodelldata = joinpath(config["main"]["input"], "dataset_vassdrag")
        tidsserie = JSON.parsefile(joinpath(sti_dynmodelldata,"tidsserier_detd.json"))
        detdseries = getelements(tidsserie, sti_dynmodelldata);
        dst = JSON.parsefile(joinpath(sti_dynmodelldata, "dataset_detd_" * watercourse * ".json"))
        detdstructure = getelements(dst);
        elements = vcat(detdseries,detdstructure)

        # Add an exogenous price area that the plants and pumps can interact with. All units are in NO5.
        for elspotname in elspotnames
            addexogenbalance!(elements, "PowerBalance_" * elspotname, "Power", "AreaPrice")
        end

        # Add dataelements for price in exogen area
        price_path = joinpath(config["main"]["input"], config["data"]["price"])
        df = CSV.read(price_path, DataFrame; header=3, decimal=',', types=Float64)
        df[:,"aar"] = cld.(1:first(size(df)), 2912) .+ 1957
        df[:,"tsnitt"] = rem.(0:(first(size(df))-1), 2912) .+ 1
        df.datetime .= getisoyearstart.(Int.(df.aar)) + Hour.((df.tsnitt.-1)*3) # TODO: Include week 53. Now ignored and flat prices.
        push!(elements, DataElement(TIMEINDEX_CONCEPT,"VectorTimeIndex","AreaPriceProfileIndex",
                Dict("Vector" => df.datetime)))
        push!(elements, DataElement(TIMEVALUES_CONCEPT,"VectorTimeValues","AreaPriceProfileValues",
                Dict("Vector" => df[:,priceseriesname])))
        push!(elements, getelement(TIMEVECTOR_CONCEPT,"RotatingTimeVector","AreaProfile",
                (TIMEINDEX_CONCEPT,"AreaPriceProfileIndex"),(TIMEVALUES_CONCEPT,"AreaPriceProfileValues")))
        addparam!(elements, "MeanSeriesIgnorePhaseinParam", "AreaPrice", 1.0, "AreaProfile")

        return Dict("elements" => elements, "detailedrescopl" => Dict())
    else
        error("$method not supported")
    end
end


In [ ]:
dataset = getdataset(config, scenarioyear)

In [ ]:
data = JulES.run_serial(config, datayear, scenarioyear, dataset)